# **DETECÇÃO DE FRAUDES EM TEMPO REAL**
Uma abordagem baseada em aprendizagem de máquina.

## Seções deste notebook

1. [Entendimento do Problema de Negócio](#entendimento-do-problema-de-negócio);
2. [Pré-processamento, Limpeza e Padronização dos Dados]() (*preprocessing*);
3. [Tratamento de Valores Faltantes]() (*data imputation*);
4. [Análise Exploratória]() (EDA);
5. [Tratamento de *outliers*]();
6. [Engenharia de Features]() (*feature engineering*);
7. [Seleção de Modelos]() (*model selection*);
8. [Seleção de Features]() (*feature selection*);
9. [Tunagem de Hiperparâmetros]() (*hyperparameter tuning*);
10. [Serialização do Modelo]() (*deploy*);
11. [Ajuste de *threshold*]();
12. [Explicabilidade do Modelo](@explicabilidade-do-modelo) (SHAP);
13. [Considerações Finais]();

## Importações e Configurações

In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

import etl.metadata as meta

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
RAW_DATA_PATH = './data/raw/creditcard.csv'

---

## Entendimento do Problema de Negócio

## Pré-processamento, Limpeza e Padronização dos Dados

O código abaixo lê os dados brutos, referentes às transações de cartão de crédito da Europa.

In [3]:
df = pd.read_csv(RAW_DATA_PATH)
df.shape

(284807, 31)

Observamos que a base possui 284.807 linhas e 31 colunas.

Abaixo, vamos dar uma olhada no cabeçalho da base:

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
df.dtypes

time                                      float64
bandeira_origem_v1                        float64
bandeira_destino_v2                       float64
tipo_transacao_v3                         float64
tipo_transacao_2_v4                       float64
banco_origem_v5                           float64
banco_destino_v6                          float64
longitude_v7                              float64
latitude_v8                               float64
periodo_do_dia_transacao_v9               float64
data_transacao_dia_da_semana_v10          float64
data_transacao_mes_v11                    float64
dist_origem_destino_v12                   float64
valor_saldo_anterior_origem_v13           float64
valor_saldo_anterior_destino_v14          float64
tipo_cartao_v15                           float64
debito_credito_v16                        float64
tempo_conta_origem_v17                    float64
tempo_conta_destino_v18                   float64
grupo_ip_origem_v19                       float64


Como mencionado anteriormente, o nome das colunas está mascarado por questões de privacidade e anonimato.

Nesse caso, os processos realizados em `etl/airflow/scripts/transform.py` referentes à limpeza dos dados, padronização e imputação de possíveis valores faltantes serão realizados por aqui, em caráter de exemplificação.

In [5]:
for col, fictional_name in meta.column_names.items():
    print(f'{col}: {fictional_name}')

V1: bandeira_origem_v1
V2: bandeira_destino_v2
V3: tipo_transacao_v3
V4: tipo_transacao_2_v4
V5: banco_origem_v5
V6: banco_destino_v6
V7: longitude_v7
V8: latitude_v8
V9: periodo_do_dia_transacao_v9
V10: data_transacao_dia_da_semana_v10
V11: data_transacao_mes_v11
V12: dist_origem_destino_v12
V13: valor_saldo_anterior_origem_v13
V14: valor_saldo_anterior_destino_v14
V15: tipo_cartao_v15
V16: debito_credito_v16
V17: tempo_conta_origem_v17
V18: tempo_conta_destino_v18
V19: grupo_ip_origem_v19
V20: pais_origem_v20
V21: pais_destino_v21
V22: grupo_cep_origem_v22
V23: grupo_cep_destino_v23
V24: data_expiracao_v24
V25: limiar_fraude_v25
V26: bandeira_cartao_origem_v26
V27: grupo_sistema_operacional_origem_v27
V28: taxa_incidencia_fraudes_no_passado_v28
Amount: Amount
Time: Time
Class: Fraud


In [6]:
# renomeando as colunas referentes ao PCA
# para nomes fictícios
df = df.rename(columns=meta.column_names)

# deixando as colunas em letras minúsculas
df.columns = [col.lower() for col in df.columns]

In [7]:
df.head()

,time,bandeira_origem_v1,bandeira_destino_v2,tipo_transacao_v3,tipo_transacao_2_v4,banco_origem_v5,banco_destino_v6,longitude_v7,latitude_v8,periodo_do_dia_transacao_v9,...,pais_destino_v21,grupo_cep_origem_v22,grupo_cep_destino_v23,data_expiracao_v24,limiar_fraude_v25,bandeira_cartao_origem_v26,grupo_sistema_operacional_origem_v27,taxa_incidencia_fraudes_no_passado_v28,amount,fraud
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Tratamento de Valores Faltantes

In [10]:
df.isnull().sum() / len(df)

time                                      0.0
bandeira_origem_v1                        0.0
bandeira_destino_v2                       0.0
tipo_transacao_v3                         0.0
tipo_transacao_2_v4                       0.0
banco_origem_v5                           0.0
banco_destino_v6                          0.0
longitude_v7                              0.0
latitude_v8                               0.0
periodo_do_dia_transacao_v9               0.0
data_transacao_dia_da_semana_v10          0.0
data_transacao_mes_v11                    0.0
dist_origem_destino_v12                   0.0
valor_saldo_anterior_origem_v13           0.0
valor_saldo_anterior_destino_v14          0.0
tipo_cartao_v15                           0.0
debito_credito_v16                        0.0
tempo_conta_origem_v17                    0.0
tempo_conta_destino_v18                   0.0
grupo_ip_origem_v19                       0.0
pais_origem_v20                           0.0
pais_destino_v21                  

In [11]:
df.isnull().sum().sum() / len(df)

0.0

## Análise Exploratória

## Separação dos Dados

In [16]:
X = df.drop('fraud', axis=1).values
y = df['fraud'].values

X.shape, y.shape
(X_train,
 X_test,
 y_train,
 y_test) = train_test_split(
    X, y,
    test_size=.3,
    shuffle=False
)

X_train.shape, y_train.shape

((199364, 30), (199364,))

## Engenharia de Features

## Seleção de Modelos

In [ ]:
clf = LazyClassifier(
    verbose=0,
    ignore_warnings=True,
    custom_metric=None
)

models, predictions = clf.fit(
    X_train,
    X_test,
    y_train,
    y_test
)

print(models)

## Tratamento de *outliers*

## Seleção de Features

## Tunagem de Hiperparâmetros

## Serialização do Modelo

## Ajuste de *threshold*

## Explicabilidade do Modelo

## Considerações Finais